In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Load the dataset
df = pd.read_excel('Activity_Dataset.xlsx')
df.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
# Separate features (Activity Title) and labels (Category)
X = df['Activity Title']
y = df['Category']

# Vectorize the text data
vectorizer = CountVectorizer(stop_words='english')
X_vectorized = vectorizer.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

In [ ]:
# Split the dataset
X_train, X_temp, y_train, y_temp = train_test_split(
    X_vectorized.toarray(), y_categorical, test_size=0.4, random_state=42
)  # 60% training data

X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)  # 20% validation and 20% testing data

In [ ]:
# Neural Network model 
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(y_categorical.shape[1], activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f'Test Accuracy: {test_accuracy}')

In [ ]:
# Function to predict the category from new input
def predict_category(input_text):
    # Transform the input text using the trained vectorizer
    input_vectorized = vectorizer.transform([input_text])
    
    # Predict using the model
    predictions = model.predict(input_vectorized.toarray())
    
    # Get the index of the category with the highest probability
    predicted_index = predictions.argmax()
    
    # Convert the index to the category name using label_encoder
    predicted_category = label_encoder.inverse_transform([predicted_index])[0]
    
    return predicted_category

# prediction
new_input = "playing tennis with my friend"
predicted_category = predict_category(new_input)
print(f"{new_input} : {predicted_category}")

In [ ]:
model.export("saved_model")

In [ ]:
# Initialize the TFLite converter
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model")

# Convert to TFLite
tflite_model = converter.convert()

# Save the TFLite model
with open("activity.tflite", "wb") as f:
    f.write(tflite_model)